In [2]:
# armazenar os documentos totais em variavel

import os
#rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/20_newsgroup'
rootdir = '/media/developer/DATA/doutorado/rec informacao/20_newsgroup'
docs = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #if os.path.basename(subdir) in ['alt.atheism', 'rec.sport.baseball']:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [8]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

#stemmer = PorterStemmer()
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
def remove_header(text):
    # Find the first occurrence of '\n\n'
    pos = text.find('\n\n')

    # Remove everything after the first '\n\n'
    if pos >= 0:
        text = text[pos+2:]
    return text
    

In [10]:
def remove_email(text):
    pattern = r'\S+@\S+'
    # Remove email addresses from the string
    text = re.sub(pattern, '', text)
    return text


In [11]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):


    #text = remove_header(text)

    #text = remove_email(text)


    # Converter tudo para letras minúsculas
    text = text.lower()
    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)
    
    # manter so texto
    #text = re.sub('[^A-Za-z ]+', '', text)

    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    #words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)
  
    return text

In [12]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    for document in docs:
        tema = document['tema']
        nome_arquivo = document['nome_arquivo']
        conteudo = document['conteudo']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
            'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [13]:
preprocessed_documents = preprocess_collection(docs)
frases = [o['conteudo'] for o in preprocessed_documents]

In [15]:
frases[0]

"xref cantaloupe.srv.cs.cmu.edu talk.abortion:120737 alt.atheism:53341 talk.religion.misc:83726 path cantaloupe.srv.cs.cmu.edu magnesium.club.cc.cmu.edu news.sei.cmu.edu cis.ohio-state.edu pacific.mps.ohio-state.edu zaphod.mps.ohio-state.edu usc sol.ctr.columbia.edu ira.uka.de germany.eu.net thoth.mchp.sni.de horus.ap.mchp.sni.de d012s658 frank frank d012s658.uucp frank o'dwyer newsgroups talk.abortion alt.atheism talk.religion.misc subject 2000 year say christian morality date apr 1993 20:00:08 gmt organization siemens-nixdorf line message-id reference nntp-posting-host d012s658.ap.mchp.sni.de article mathew writes frank d012s658.uucp frank o'dwyer writes article mathew mantis.co.uk mathew writes ask think wrong relativism correct misconception well cut chase admit find least attractive realtivism elevates heinous level good saying effect good n't good thus terrorist elevated level man peace complete nonsense relativism mean saying absolute standard morality mean saying standard moral

In [16]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in frases]

bm25 = BM25Okapi(tokenized_corpus)

In [39]:
# armazenar os documentos totais em variavel

import os
#rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/mini_newsgroups'
rootdir = '/media/developer/DATA/doutorado/rec informacao/mini_newsgroups'

docs_consultas = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs_consultas.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [40]:
dirs_abs = []
for subdir, dirs, files in os.walk(rootdir):
    if os.path.basename(subdir) != '20_newsgroup':
        dirs_abs.append(os.path.basename(subdir))

temas = {}
for d in dirs_abs:
    temas[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(temas)    

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'alt.atheism': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.graphics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.os.ms-windows.misc': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.ibm.pc.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.mac.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.windows.x': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'misc.forsale': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.autos': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.motorcycles': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.baseball': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.hockey': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.crypt': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.electronics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 

In [41]:
def calcula_map(ranking, tema):
    acertos = 0
    maps = []
    for i, doc in enumerate(ranking):
        if tema in doc:
            acertos +=1
            maps.append(acertos/(i+1))
    if len(maps) > 0:
        return mean(maps)
    else:
        print("não achou nada")
        return 0.0

In [29]:
doc_consultas_preproc = preprocess_collection(docs_consultas)

In [42]:
doc_consultas_preproc[0]['conteudo']

"newsgroups alt.atheism path cantaloupe.srv.cs.cmu.edu das-news.harvard.edu noc.near.net howland.reston.ans.net zaphod.mps.ohio-state.edu darwin.sura.net haven.umd.edu uunet mnemosyne.cs.du.edu nyx jcopelan jcopelan nyx.cs.du.edu one subject message-id sender usenet mnemosyne.cs.du.edu netnews admin account organization salvation army draft board reference date fri apr 13:38:30 gmt line article geoff east.sun.com writes posting provoked checking save file memorable post first captured ken arromdee feb 1990 subject atheist article 473 question article 53766 average article day last three year others noted current posting rate kill file depressing large among posting saved early day article following notable loren sunlight.llnl.gov loren petrich jchrist nazareth.israel.rel jesus christ nazareth mrc tomobiki-cho.cac.washington.edu mark crispin perry apollo.hp.com jim perry lippard uavax0.ccit.arizona.edu james lippard minsky media.mit.edu marvin minsky interesting bunch wonder n't hear ad

In [37]:
tokenized_query = doc_consultas_preproc[101]['conteudo'].split(" ")

In [38]:
bm25.get_top_n(tokenized_query, frases, n=100)

["newsgroups comp.graphics path cantaloupe.srv.cs.cmu.edu magnesium.club.cc.cmu.edu news.sei.cmu.edu fs7.ece.cmu.edu europa.eng.gtefsd.com howland.reston.ans.net usc elroy.jpl.nasa.gov decwrl hal.com olivea charnel rat koko.csustan.edu altair.csustan.edu rsc rsc altair.csustan.edu steve cunningham subject acm siggraph registration problem message-id summary fixed sender steve cunningham organization csu stanislaus reference date thu apr 02:35:08 gmt line zyda cs.nps.navy.mil michael zyda note word warning registering siggraph '93 received registration form back mail envelope marked return sender moved left address closed box address used register siggraph '93 one printed registration form acm siggraph '93 box 95316 chicago 60694-5316 printed envelope best printing honest evidently siggraph '93 skipped town moved ended faxing registration 312-321-6876. hope number real michael zyda problem called people handle box problem happened time ago caught almost instantly registration going addr

In [43]:
from statistics import mean

from IPython.display import clear_output

# consultando
p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""

for c, doc_consulta in enumerate(doc_consultas_preproc):
    clear_output(wait=True)
    print(c)
    #ranking = similaridade(doc_consulta['conteudo'], preprocessed_documents, idfs)
    tokenized_query = doc_consulta['conteudo'].split(" ")
    ranking = bm25.get_top_n(tokenized_query, frases, n=100)
    acertos = 0
    
    map_f.append(calcula_map(ranking, doc_consulta['tema']))
    
    for i, doc in enumerate(ranking):
        #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
        #if doc[1] == doc_consulta['tema']:
        
        if doc_consulta['tema'] in doc:
            #print(f"{doc_consulta['tema']} e {doc}")
            acertos+=1
        if i+1 == 10:
            p10.append(acertos/10.0)            
        if i+1 == 20:
            p20.append(acertos/20.0)
        if i+1 == 50:
            p50.append(acertos/50.0)
        if i+1 == 100:
            p100.append(acertos/100.0) 
        #map.append(calcula_map(ranking, doc_consulta['tema']))
    if c > 0:
        if tema_atual != doc_consulta['tema'] or c == len(doc_consultas_preproc) - 1:
            temas[tema_atual]['p10'] = mean(p10)
            temas[tema_atual]['p20'] = mean(p20)
            temas[tema_atual]['p50'] = mean(p50)
            temas[tema_atual]['p100'] = mean(p100)
            temas[tema_atual]['map'] = mean(map_f)
            p10 = []
            p20 = []
            p50 = []
            p100 = []
            map = []

    tema_atual = doc_consulta['tema']
    if c > 202:
        break    


0


TypeError: list indices must be integers or slices, not str

In [34]:
temas

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'alt.atheism': {'p10': 0.8910891089108911,
  'p20': 0.49554455445544554,
  'p50': 0.416039603960396,
  'p100': 0.2972277227722772,
  'map': 0.6006383806932881},
 'comp.graphics': {'p10': 0.0,
  'p20': 0.0495,
  'p50': 0.0202,
  'p100': 0.020300000000000002,
  'map': 0.3198408344212838},
 'comp.os.ms-windows.misc': {'p10': 0.0,
  'p20': 0.0,
  'p50': 0.04,
  'p100': 0.050100000000000006,
  'map': 0.22973015588710635},
 'comp.sys.ibm.pc.hardware': {'p10': 0.0,
  'p20': 0.0005,
  'p50': 0.04,
  'p100': 0.059699999999999996,
  'map': 0.18643666665971134},
 'comp.sys.mac.hardware': {'p10': 0.0,
  'p20': 0.0495,
  'p50': 0.0396,
  'p100': 0.0298,
  'map': 0.15936297520019257},
 'comp.windows.x': {'p10': 0.0,
  'p20': 0.001,
  'p50': 0.0008,
  'p100': 0.011,
  'map': 0.135801043190376},
 'misc.forsale': {'p10': 0.0,
  'p20': 0.099,
  'p50': 0.0792,
  'p100': 0.109,
  'map': 0.13023473694978882},
 'rec.autos': {'p10': 0.

In [35]:
import json
with open('data_lemma_bm25.json', 'w') as f:
    json.dump(temas, f)

In [36]:
import pandas as pd

df = pd.DataFrame(temas)
df =df.transpose()
df.drop('mini_newsgroups')


,p10,p20,p50,p100,map
alt.atheism,0.891089,0.495545,0.41604,0.297228,0.600638
comp.graphics,0.000000,0.049500,0.02020,0.020300,0.319841
comp.os.ms-windows.misc,0.000000,0.000000,0.04000,0.050100,0.229730
comp.sys.ibm.pc.hardware,0.000000,0.000500,0.04000,0.059700,0.186437
comp.sys.mac.hardware,0.000000,0.049500,0.03960,0.029800,0.159363
comp.windows.x,0.000000,0.001000,0.00080,0.011000,0.135801
misc.forsale,0.000000,0.099000,0.07920,0.109000,0.130235
rec.autos,0.000000,0.000000,0.00040,0.010300,0.115847
rec.motorcycles,0.000000,0.001000,0.04120,0.040700,0.107828
rec.sport.baseball,0.000000,0.099000,0.15840,0.108900,0.110887


In [ ]:
df